In [7]:
import pandas as pd

In [8]:
df = pd.read_parquet('../do_not_commit/Datasets/Persist_WorkstationStatus_Data.pq')

In [9]:
df['RWB_EFFECTIVE_DATE'].max()

'2023-07-02'


# last_hw_scan_datetime


In [10]:
# Datetime of the last hardware scan. Grab this from LastHWScan from WorkstationStatus_Data.
last_time = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE']).agg({'LastHWScan':'min'}).reset_index()
last_time

,MachineID,RWB_EFFECTIVE_DATE,LastHWScan
0,16783564,2023-01-19,2023-01-18 14:43:36.000
1,16783564,2023-01-20,2023-01-19 12:44:36.000
2,16783564,2023-01-21,2023-01-21 06:52:59.000
3,16783564,2023-01-22,2023-01-22 07:16:54.000
4,16783564,2023-01-23,2023-01-23 05:09:56.000
...,...,...,...
1064877,16819023,2023-07-01,2023-06-30 12:42:05.000
1064878,16819023,2023-07-02,2023-07-02 05:46:22.000
1064879,16819024,2023-07-01,2023-06-30 19:31:29.000
1064880,16819024,2023-07-02,2023-07-02 02:30:37.000



# last_hw_scan_age


In [11]:
# Age in days of the last hardware scan. Calculate this from LastHWScan from WorkstationStatus_Data.

In [12]:
last_hw_scan_days = df.groupby(['MachineID', 'RWB_EFFECTIVE_DATE'])['LastHWScan'].agg('min').reset_index()

In [13]:
last_hw_scan_days['RWB_EFFECTIVE_DATE'] = pd.to_datetime(last_hw_scan_days['RWB_EFFECTIVE_DATE']).dt.date
last_hw_scan_days['LastHWScan'] = pd.to_datetime(last_hw_scan_days['LastHWScan'])

In [14]:
last_hw_scan_days['LastHWScan_Date'] = last_hw_scan_days['LastHWScan'].apply(lambda x: x.date())
last_hw_scan_days['LastHWScan_Days'] = last_hw_scan_days['RWB_EFFECTIVE_DATE'] - last_hw_scan_days['LastHWScan_Date']

In [15]:
last_hw_scan_days['LastHWScan_Days'] = last_hw_scan_days['LastHWScan_Days'].apply(lambda x: x.days)
last_hw_scan_days

,MachineID,RWB_EFFECTIVE_DATE,LastHWScan,LastHWScan_Date,LastHWScan_Days
0,16783564,2023-01-19,2023-01-18 14:43:36,2023-01-18,1
1,16783564,2023-01-20,2023-01-19 12:44:36,2023-01-19,1
2,16783564,2023-01-21,2023-01-21 06:52:59,2023-01-21,0
3,16783564,2023-01-22,2023-01-22 07:16:54,2023-01-22,0
4,16783564,2023-01-23,2023-01-23 05:09:56,2023-01-23,0
...,...,...,...,...,...
1064877,16819023,2023-07-01,2023-06-30 12:42:05,2023-06-30,1
1064878,16819023,2023-07-02,2023-07-02 05:46:22,2023-07-02,0
1064879,16819024,2023-07-01,2023-06-30 19:31:29,2023-06-30,1
1064880,16819024,2023-07-02,2023-07-02 02:30:37,2023-07-02,0


In [16]:
last_time['RWB_EFFECTIVE_DATE'] = last_time['RWB_EFFECTIVE_DATE'].astype('str')
last_hw_scan_days['RWB_EFFECTIVE_DATE'] = last_hw_scan_days['RWB_EFFECTIVE_DATE'].astype('str')

In [17]:
combined = last_time.merge(last_hw_scan_days[['MachineID', 'RWB_EFFECTIVE_DATE', 'LastHWScan_Days']], on=['MachineID', 'RWB_EFFECTIVE_DATE'])
combined

,MachineID,RWB_EFFECTIVE_DATE,LastHWScan,LastHWScan_Days
0,16783564,2023-01-19,2023-01-18 14:43:36.000,1
1,16783564,2023-01-20,2023-01-19 12:44:36.000,1
2,16783564,2023-01-21,2023-01-21 06:52:59.000,0
3,16783564,2023-01-22,2023-01-22 07:16:54.000,0
4,16783564,2023-01-23,2023-01-23 05:09:56.000,0
...,...,...,...,...
1064877,16819023,2023-07-01,2023-06-30 12:42:05.000,1
1064878,16819023,2023-07-02,2023-07-02 05:46:22.000,0
1064879,16819024,2023-07-01,2023-06-30 19:31:29.000,1
1064880,16819024,2023-07-02,2023-07-02 02:30:37.000,0


In [18]:
combined.to_parquet('../do_not_commit/FeatureDatasets/workstationstatus_features.pq')

In [19]:
# df.groupby().agg(lambda x: pd.Series(dict{}))